In [25]:
import os

from sklearn.cross_validation import KFold
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn import metrics
from scipy import stats

from assistments_workbench.data_converters.generate_BKT_data import BKTConverter
from assistments_workbench.models.model import classification_model
from assistments_workbench.config_reader import config
from BKT_dot_py.BKT import BKT

In [26]:
col_mapping = {'user_id': 1, 'sequence_id' : 2, 'problem_id' : 3, 'correct': 4}

In [27]:
input_data = os.path.join(config.get('localfiles', 'data_path'), 'sql_data_3.csv')
output_data = os.path.join(config.get('localfiles', 'data_path'), 'bkt', 'nnn.txt')

In [28]:
converter = BKTConverter(input_data, output_data, col_mapping)

In [29]:
bkt_data = converter.convert()

In [30]:
for ps_id in bkt_data.keys():
    print ps_id
    auc, r2, rmse = [], [], []
    this_ps_data = bkt_data[ps_id]
    data_len = len(this_ps_data)
    kf = KFold(data_len, n_folds = 5, shuffle = True)
    for train, test in tqdm(kf):
        train_data = [this_ps_data.values()[i] for i in train]
        test_data = [this_ps_data.values()[i] for i in test]
        bkt = BKT(step = 0.1, bounded = False, best_k0 = False)
        bkt.fit(train_data)
        error, predictions =  bkt.predict(test_data)
        
        all_test_data = [item for sublist in test_data for item in sublist]
        all_predictions = [item for sublist in predictions for item in sublist]
        
        auc.append(metrics.roc_auc_score(all_test_data, all_predictions))
        r, _ = stats.pearsonr(all_test_data, all_predictions)
        r2.append(r**2)
        rmse.append(metrics.mean_squared_error(all_test_data, all_predictions)**0.5)
    print 'AUC: ', np.mean(auc)
    print 'r2:', np.mean(r**2)
    print 'rmse:', np.mean(rmse)
        
            

37055


AUC:  0.53622363611
r2: 4.17318560119e-05
rmse: 0.441133792065
10195


AUC:  0.611294033393
r2: 0.0233840326844
rmse: 0.453244413776
37570


 52%|████████████████████████████████████████                                     | 5205/10000 [03:57<03:01, 26.47it/s]

KeyboardInterrupt: 